# =========== ANÁLISIS DE MÉTRICAS ===========

In [107]:
import pandas as pd
import numpy as np
import os

files_path = "data/"

# Eventos buscados para cada una de las metricas
eventsOfMetrics = [["GAME_START", "GAME_END", "JUMP_START", "JUMP_END"], 
                   ["GAME_START", "GAME_END", "PLAYER_MOVE", "JUMP_END"], 
                   ["GAME_START", "GAME_END", "JUMP_START", "JUMP_END"]
                   ] 

numPlatforms = 24

In [108]:
# Devuelve los datos leidos
def ReadFile(archivo):
    try:
        # Intenta leer el archivo como JSON
        data = pd.read_json(archivo)
        # Devuelve una lista de diccionarios
        return data.to_dict(orient='records')
    except Exception as e:
        print(f"No se pudo leer el archivo {archivo}:", e)
        return None, None

In [109]:
# Filtra los eventos que se necesitan para la metrica que se va a medir
def FilterEventsOfMetric(data, events):

    #Comprobar que es una lista de diccionarios
    if isinstance(data, list):      
        filtered_events = [event for event in data if event.get('eventType') in events]
        if filtered_events:
            sorted_events = sorted(filtered_events, key=lambda x: x.get('timeStamp'))
            return sorted_events
        
        else:
            print("No se encontraron eventos con los tipos especificados en el diccionario JSON.")
            return None
        

### Número de fallos hasta llegar a la P4 por primera vez. Se considera fallo cuando el pez pasa de la plataforma ‘i’ a cualquier plataforma menor que ‘i’.

In [110]:
# Devuelve una lista con los fallos antes de pasar P4, cada valor de la lista es una partida
def FirstMetric(events):

    failures_before_P4 = 0
    games = []
    p4Passed = False

    game_start = None
    jump_start = None
    game_end = None
    jump_end = None

    for event in events:
        event_type = event.get('eventType')

        if event_type == 'GAME_START' and game_start is None:
            game_start = event

        elif event_type == 'JUMP_START':

            if game_end is not None:
                game_start = None
                game_end = None
            elif jump_start is None:
                jump_start = event

        elif event_type == 'GAME_END':

            game_end = event
            game_start = None

            # Si acaba una partida y ha habido al menos un fallo lo guardamos 
            if failures_before_P4 != 0:
                games.append(failures_before_P4)
                failures_before_P4 = 0
                p4Passed = False

        elif event_type == 'JUMP_END':

            if jump_start is not None:
                jump_end = event
                platformIdJumpStart = int(jump_start.get('platformId'))
                platformIdJumpEnd = int(jump_end.get('platformId'))

                # Reseteamos los eventos de saltar
                jump_start = None
                jump_end = None

                # Si ya se ha llegado a la P4 ya no se cuenta más veces en esta partida
                if platformIdJumpEnd >= 4:
                    p4Passed = True
                    game_start = None
                    game_end = None
                    continue

                # Si hay fallo 
                if platformIdJumpEnd < platformIdJumpStart and p4Passed is False:
                    failures_before_P4 += 1
    
    return games


### Tasa de recolocación: plataformas en las que el usuario utiliza “WASD” para moverte al menos una vez entre las plataformas pisadas superiores a P1.

In [111]:
# Devuelve una lista de arrays que contiene las veces que te has movido en cada plataforma, cada array de la lista es una partida
def SecondMetric(events):
    
    movesInEachPlatform = []
    games = []

    game_start = None
    game_end = None

    for event in events:
        event_type = event.get('eventType')
        
        if event_type == 'GAME_START' and game_start is None:
            game_start = event

        elif event_type == 'PLAYER_MOVE':

            if game_end is not None:
                game_start = None
                game_end = None
            else:
                platformId = int(event.get('platformId'))

                if platformId != 1:
                    movesInEachPlatform[-1] += 1

        
        elif event_type == 'JUMP_END':
            platformId = int(event.get('platformId'))

            if platformId != 1:
                movesInEachPlatform.append(0)


        elif event_type == 'GAME_END':

            game_end = event
            game_start = None

            # Si acaba una partida y ha habido al menos un movimiento lo guardamos 
            if any(movesInEachPlatform):
                moves = np.count_nonzero(movesInEachPlatform)
                games.append(moves/len(movesInEachPlatform))
                movesInEachPlatform = []
    
    return games

### Número de fallos de las plataformas P4, P15 y P22 antes de superarlas por primera vez. Hemos elegido estas plataformas porque en ellas es necesario el uso de los rebotes.

In [112]:
# Devuelve una lista de eventos donde cada elemento es otra lista con 3 elementos, uno por cada plataforma a medir 
def ThirdMetric(events):

    rebound_platforms = [4,15,22]
    rebound_platforms_aux = rebound_platforms
    failures_in_rebound_platforms = np.zeros(3)

    index = 0

    games = []

    game_start = None
    jump_start = None
    game_end = None
    jump_end = None

    for event in events:
        event_type = event.get('eventType')

        if event_type == 'GAME_START' and game_start is None:
            game_start = event

        elif event_type == 'JUMP_START':

            if game_end is not None:
                game_start = None
                game_end = None
            elif jump_start is None:
                jump_start = event

        elif event_type == 'GAME_END':

            game_end = event
            game_start = None

            #print(failures_in_rebound_platforms)

            # Si acaba una partida y ha habido al menos un fallo lo guardamos 
            if any(failures_in_rebound_platforms):
                games.append(failures_in_rebound_platforms)
                failures_in_rebound_platforms = np.zeros(3)
                index = 0
                rebound_platforms = rebound_platforms_aux

        elif event_type == 'JUMP_END':

            if jump_start is not None:
                jump_end = event
                platformIdJumpStart = int(jump_start.get('platformId'))
                platformIdJumpEnd = int(jump_end.get('platformId'))

                # Reseteamos los eventos de saltar
                jump_start = None
                jump_end = None

                for reboundPlatform in rebound_platforms:
                    if platformIdJumpStart == reboundPlatform:

                        if platformIdJumpEnd < platformIdJumpStart:
                            failures_in_rebound_platforms[index] += 1
                            
                        elif platformIdJumpEnd > platformIdJumpStart:
                            del rebound_platforms[0]
                            index += 1

                    elif platformIdJumpStart < reboundPlatform and platformIdJumpEnd > reboundPlatform:
                            del rebound_platforms[0]
                            index += 1
                

                #display(rebound_platforms)

                if not rebound_platforms:
                    game_start = None
                    game_end = None
                    continue

    
    return games

In [113]:
# Por cada archivo dentro de data/
for file in os.listdir(files_path):
    file_path = os.path.join(files_path, file)

    if os.path.isfile(file_path):

        data = ReadFile(file_path)

        i = 0

        print("\n \n")
        print("=========================================")
        print(" === EVENTOS DE " + file + " ===")
        print("=========================================")


        for event in eventsOfMetrics:
            
            filteredEvents = FilterEventsOfMetric(data, event)

            if i == 0:
                print("\n")
                print("=========================================================================")
                print("1º MÉTRICA:  FALLOS ANTES DE PASAR P4 POR PRIMERA VEZ")
                print("=========================================================================")
                
                display(FirstMetric(filteredEvents))

                print("Descripcion del valor obtenido:")
                print("===============================")
                print("El siguiente valor indica cuantos fallos a tenido cada uno de los jugadores en la primera partida registrada antes de alcanzar la plataforma 4."
                        + "\n La media de todos esos valores es la metrica que se muestra a continuación.")

            elif i == 1:
                print("\n")
                print("=========================================================================")
                print("2º MÉTRICA: TASA DE RECOLOCACIÓN")
                print("=========================================================================")
                
                display(SecondMetric(filteredEvents))

                print("Descripción del valor obtenido:")
                print("===============================")
                print("Media de la tasa de recolocación. "
                      + " \n Se define tasa de recolocación como plataformas en las que el usuario utiliza “WASD” para moverte al menos una vez entre las plataformas pisadas superiores a P1. La tasa de recolocación se mide por partida.")
                print("Es decir, se divide el número de veces que el jugador ha movido a su personaje en cualquier plataforma, entre el número de plataformas que ha recorrido en total en esta partida"
                      + " \n Por lo tanto, se obtiene un numero decimal entre 0 y 1 que muestra la cantidad de veces que este jugador ha usado el movimiento del personaje en esta partida concreta."
                      + " \n La media de los valores de todas las partidas registradas es la métrica que se muestra a continuación.")

            elif i == 2:
                print("\n")
                print("=========================================================================")
                print("3º METRICA: NUMERO DE FALLOS ANTES DE PASAR POR PRIMERA VEZ P4, P15 y P22")
                print("=========================================================================")

                display(ThirdMetric(filteredEvents))

                print("Descripción del valor obtenido:")
                print("===============================")
                print("Media de fallos de las plataformas P4, P15 y P22 antes de superarlas por primera vez.")
                print("Es decir, para cada una de las plataformas P4, P15 y P22, se calculan los fallos que ha tenido un jugador antes de superarlas por primera vez, con lo que se saca una media de fallos por jugador"
                      + "\n Cada jugador, generara entonces un valor por cada una de las tres plataformas."
                        + "\n La media de los valores de todos los jugadores)")
                
            i += 1



 

 === EVENTOS DE eventsRayo.json ===


1º MÉTRICA:  FALLOS ANTES DE PASAR P4 POR PRIMERA VEZ


[]

Descripcion del valor obtenido:
El siguiente valor indica cuantos fallos a tenido cada uno de los jugadores en la primera partida registrada antes de alcanzar la plataforma 4.
 La media de todos esos valores es la metrica que se muestra a continuación.


2º MÉTRICA: TASA DE RECOLOCACIÓN


[0.8152173913043478,
 0.5652173913043478,
 0.14285714285714285,
 0.045454545454545456,
 0.3888888888888889,
 0.4,
 0.05,
 0.09090909090909091,
 0.2571428571428571,
 0.14285714285714285,
 0.08695652173913043,
 0.34615384615384615,
 0.38461538461538464,
 0.125,
 0.14285714285714285,
 0.043478260869565216,
 0.029411764705882353,
 0.045454545454545456,
 0.2608695652173913,
 0.23076923076923078,
 0.18181818181818182,
 0.38461538461538464,
 0.05714285714285714,
 0.16666666666666666,
 0.3333333333333333,
 0.23076923076923078,
 0.01694915254237288,
 0.4,
 0.02857142857142857,
 0.03225806451612903,
 0.14285714285714285,
 0.03571428571428571,
 0.2,
 0.05263157894736842,
 0.2,
 0.09090909090909091,
 0.125,
 0.1,
 0.3157894736842105,
 0.1,
 0.1,
 0.15384615384615385,
 0.027777777777777776,
 0.1111111111111111,
 0.058823529411764705,
 0.15625,
 0.14814814814814814,
 0.043478260869565216]

Descripción del valor obtenido:
Media de la tasa de recolocación.  
 Se define tasa de recolocación como plataformas en las que el usuario utiliza “WASD” para moverte al menos una vez entre las plataformas pisadas superiores a P1. La tasa de recolocación se mide por partida.
Es decir, se divide el número de veces que el jugador ha movido a su personaje en cualquier plataforma, entre el número de plataformas que ha recorrido en total en esta partida 
 Por lo tanto, se obtiene un numero decimal entre 0 y 1 que muestra la cantidad de veces que este jugador ha usado el movimiento del personaje en esta partida concreta. 
 La media de los valores de todas las partidas registradas es la métrica que se muestra a continuación.


3º METRICA: NUMERO DE FALLOS ANTES DE PASAR POR PRIMERA VEZ P4, P15 y P22


...

Descripcion del valor obtenido:
El siguiente valor indica cuantos fallos a tenido cada uno de los jugadores en la primera partida registrada antes de alcanzar la plataforma 4.
 La media de todos esos valores es la metrica que se muestra a continuación.


2º MÉTRICA: TASA DE RECOLOCACIÓN


[]

Descripción del valor obtenido:
Media de la tasa de recolocación.  
 Se define tasa de recolocación como plataformas en las que el usuario utiliza “WASD” para moverte al menos una vez entre las plataformas pisadas superiores a P1. La tasa de recolocación se mide por partida.
Es decir, se divide el número de veces que el jugador ha movido a su personaje en cualquier plataforma, entre el número de plataformas que ha recorrido en total en esta partida 
 Por lo tanto, se obtiene un numero decimal entre 0 y 1 que muestra la cantidad de veces que este jugador ha usado el movimiento del personaje en esta partida concreta. 
 La media de los valores de todas las partidas registradas es la métrica que se muestra a continuación.


3º METRICA: NUMERO DE FALLOS ANTES DE PASAR POR PRIMERA VEZ P4, P15 y P22


[array([1., 0., 0.])]

Descripción del valor obtenido:
Media de fallos de las plataformas P4, P15 y P22 antes de superarlas por primera vez.
Es decir, para cada una de las plataformas P4, P15 y P22, se calculan los fallos que ha tenido un jugador antes de superarlas por primera vez, con lo que se saca una media de fallos por jugador
 Cada jugador, generara entonces un valor por cada una de las tres plataformas.
 La media de los valores de todos los jugadores)
